In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

# Assuming your file is in Excel format (xlsx or xls)
file_path = 'Monthly_ave_realistic_cleaner_Q_cfs_Station_MER.xlsx'

# Read data from Sheet1 and Sheet2
df_sheet1 = pd.read_excel(file_path, sheet_name='Sheet1')
df_sheet2 = pd.read_excel(file_path, sheet_name='Sheet2')

# Extracting the MonthlyAvgFlow and HistAvgFlow data
monthly_avg_flow = df_sheet1['MonthlyAvgFlow'].values
hist_avg_flow = df_sheet2['HistAvgFlow'].values

# Perform quantile mapping on HistAvgFlow
def quantile_mapping(reference_data, model_data):
    # Calculate empirical CDFs for both datasets
    model_sorted = np.sort(model_data)
    ref_sorted = np.sort(reference_data)
    model_cdf = np.arange(1, len(model_data) + 1) / len(model_data)
    ref_cdf = np.arange(1, len(reference_data) + 1) / len(reference_data)

    # Interpolation function: model CDF to reference data
    interp_func = interp1d(model_cdf, ref_sorted, bounds_error=False, fill_value="extrapolate")
    
    # Map model data to quantile values of reference data
    mapped_values = interp_func(np.argsort(np.argsort(model_data)) / len(model_data))

    return mapped_values


# Assuming the last 22 years for the reference data
data_22_years = monthly_avg_flow[-22*12:]  # adjust based on your data structure
qm_values_hist_avg_flow = quantile_mapping(data_22_years, hist_avg_flow)

# Adding the quantile-mapped values as a new column to the DataFrame of Sheet2
df_sheet2['QM_HistAvgFlow'] = qm_values_hist_avg_flow

# Function to calculate and plot CDF
def plot_cdf(data, label, color):
    sorted_data = np.sort(data)
    yvals = np.arange(1, len(sorted_data)+1) / float(len(sorted_data))
    plt.plot(sorted_data, yvals, label=label, color=color)

# Plotting original CDFs
plt.figure(figsize=(10, 6))
plot_cdf(hist_avg_flow, "HistAvgFlow Original", "blue")
plot_cdf(monthly_avg_flow, "MonthlyAvgFlow", "green")
plt.title("Original CDFs")
plt.xlabel("Data Values")
plt.ylabel("Cumulative Probability")
plt.legend()
plt.grid(True)
plt.show()

# Apply quantile mapping
qm_values_hist_avg_flow = quantile_mapping(monthly_avg_flow, hist_avg_flow)

# Plot CDFs after quantile mapping
plt.figure(figsize=(10, 6))
plot_cdf(hist_avg_flow, "HistAvgFlow Original", "blue")
plot_cdf(qm_values_hist_avg_flow, "HistAvgFlow Quantile-Mapped", "red")
plt.title("CDFs After Quantile Mapping")
plt.xlabel("Data Values")
plt.ylabel("Cumulative Probability")
plt.legend()
plt.grid(True)
plt.show()


# Save the updated DataFrame (Sheet2) back to an Excel file
df_sheet2.to_excel('Updated_Monthly_ave_realistic_cleaner_Q_cfs_Station_MER.xlsx', index=False)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d


file_path = 'Monthly_ave_realistic_cleaner_Q_cfs_Station_MER_L.xlsx'


df_sheet1 = pd.read_excel(file_path, sheet_name='Sheet1')
df_sheet2 = pd.read_excel(file_path, sheet_name='Sheet2')


monthly_avg_flow = df_sheet1['MonthlyAvgFlow'].values
hist_avg_flow = df_sheet2['HistAvgFlow'].values


def quantile_mapping(reference_data, model_data):
    # Sort both datasets
    ref_sorted = np.sort(reference_data)
    model_sorted = np.sort(model_data)

    # Calculate quantiles for both datasets
    ref_quantiles = np.linspace(0, 1, len(ref_sorted))
    model_quantiles = np.linspace(0, 1, len(model_sorted))

    # Create an interpolation function from model data quantiles to reference data quantiles
    interp_to_ref_quantiles = interp1d(model_sorted, model_quantiles, bounds_error=False, fill_value="extrapolate")
    
    # Map model data values to reference data quantiles, then to reference data sorted values
    ref_data_quantiles = interp_to_ref_quantiles(model_data)
    mapped_values = np.interp(ref_data_quantiles, ref_quantiles, ref_sorted)

    return mapped_values




#data_22_years = monthly_avg_flow[-22*12:]  
qm_values_hist_avg_flow = quantile_mapping(data_22_years, hist_avg_flow)

# Ensure the length matches the length of 'hist_avg_flow'
assert len(qm_values_hist_avg_flow) == len(hist_avg_flow)

# Assign the quantile-mapped values to the DataFrame
df_sheet2['QM_HistAvgFlow'] = qm_values_hist_avg_flow


def plot_cdf(data, label, color):
    sorted_data = np.sort(data)
    yvals = np.arange(1, len(sorted_data)+1) / float(len(sorted_data))
    plt.plot(sorted_data, yvals, label=label, color=color)


plt.figure(figsize=(10, 6))
plot_cdf(hist_avg_flow, "HistAvgFlow Original", "blue")
plot_cdf(monthly_avg_flow, "MonthlyAvgFlow", "green")
plt.title("Original CDFs")
plt.xlabel("Data Values")
plt.ylabel("Cumulative Probability")
plt.legend()
plt.grid(True)
plt.show()


qm_values_hist_avg_flow = quantile_mapping(monthly_avg_flow, hist_avg_flow)

plt.figure(figsize=(10, 6))
plot_cdf(hist_avg_flow, "HistAvgFlow Original", "blue")
plot_cdf(qm_values_hist_avg_flow, "HistAvgFlow Quantile-Mapped", "red")
plt.title("CDFs After Quantile Mapping")
plt.xlabel("Data Values")
plt.ylabel("Cumulative Probability")
plt.legend()
plt.grid(True)
plt.show()




df_sheet2.to_excel('Updated_Monthly_ave_realistic_cleaner_Q_cfs_Station_MER_L.xlsx', index=False)